<a href="https://colab.research.google.com/github/Anikate-De/TensorFlow-Deep-Learning/blob/main/NLP%20Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import tensorflow as tf

In [30]:
!nvidia-smi

Wed Dec 27 12:53:41 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0              27W /  70W |    139MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [31]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-12-27 12:53:41--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.145.207, 173.194.69.207, 173.194.79.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.145.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  1.31MB/s    in 0.4s    

2023-12-27 12:53:42 (1.31 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [32]:
!unzip nlp_getting_started.zip

Archive:  nlp_getting_started.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


## Vizualize the dataset

In [33]:
import pandas as pd

train_df, test_df = pd.read_csv('train.csv'), pd.read_csv('test.csv')

In [34]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [35]:
train_df_shuffled = train_df.sample(frac=1, random_state=4)
train_df_shuffled.head()

,id,keyword,location,text,target
7239,10367,weapons,West,WWII Memories Plus Lessons of Hiroshima We Sti...,1
484,700,attacked,"Los Angeles, CA",@envw98 @NickCoCoFree @JulieDiCaro @jdabe80 I ...,0
861,1243,blood,NaN,My blood pressure is through the roof I don't ...,0
5602,7992,razed,"Erie, PA",today will be another dualcom with @Im_Razed !...,0
3536,5056,eyewitness,"Orlando, Fl",WFTV Eyewitness News: FBI: Man who stole US se...,0


In [36]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [37]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [38]:
len(train_df), len(test_df)

(7613, 3263)

In [39]:
# Visualise random training data

import random
random_index = random.randint(0, len(train_df) - 5)

for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print('Target:', 'not a' if target==1 else 'a', 'real disaster')
  print('Text:', text, '----', sep = '\n')


Target: not a real disaster
Text:
Hundreds feared drowned after another Mediterranean asylum seeker boat sinking http://t.co/zsYkzj2bzG
----
Target: not a real disaster
Text:
Traffic accident N CABRILLO HWY/MAGELLAN AV MIR (08/06/15 11:03:58)
----
Target: not a real disaster
Text:
@colinhoffman29 I hope he does. And I hope you die in the explosion too
----
Target: not a real disaster
Text:
#refugees of #nuclear disaster in western #Japan will be beyond 500 thousand at least. #AFP #guardian #WPS #NYTimes
----
Target: a real disaster
Text:
ColnHarun: Breaking news! Unconfirmed! I just heard a loud bang nearby. in what appears to be a blast of wind from my neighbour's ass.
----


In [40]:
from sklearn.model_selection import train_test_split

In [41]:
train_tweets, val_tweets, train_labels, val_labels = train_test_split(
    train_df_shuffled['text'].to_numpy(),
    train_df_shuffled['target'].to_numpy(),
    test_size=0.1,
    random_state=4,
)

In [42]:
len(train_tweets), len(train_labels), len(val_tweets), len(val_labels)

(6851, 6851, 762, 762)

In [43]:
train_tweets[:10], train_labels[:10]

(array(["nflexpertpicks: Michael Floyd's hand injury shouldn't devalue his fantasy stock: Michael Floyd's damaged digits won... \x89Û_",
        "@TennoAtax I hand you a glass of water and sit down. 'I was rescued by a Tenno. I've lived in my ship around the dojo for about a year...'",
        "@PerkPearl that's just not on. I'd be traumatised are you OK? The car has gone and now for #GBBO and relax.....",
        '@anellatulip and put the taint there and that all that the magisters did was to open the gates and let the blight get away from it',
        "'Suicide bombing at [location named]...' #premonitions http://t.co/iIkSsJGBDn",
        '1.43 earthquake occurred near Mount St. Helens area Washington at 09:36 UTC! #earthquake http://t.co/2xMdiDGpnr',
        "'American Weapons and Support Are Fueling a Bloody Air War in Yemen' http://t.co/7aGeAkVn2x",
        '@aptly_engineerd There is no such curfew.',
        'Guaranteed been bitten by some mutant mosquito my ankle has blown up. L

## Convert Text into Numbers

In [44]:
round(sum([len(i.split()) for i in train_tweets])/ len(train_tweets))

15

In [45]:
max_vocab_length = 10000
max_length = 15

from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    output_sequence_length = max_length,

)

In [46]:
text_vectorizer.adapt(train_tweets)

In [47]:
sample = 'There is a flood on my street!'

text_vectorizer([sample])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 75,   9,   3, 220,  11,  13, 802,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [48]:
sample = random.choice(train_tweets)

print(sample)
text_vectorizer([sample])


Photo: prettyboyshyflizzy: Lol she trapped them into that so beautifully http://t.co/FKXCsztezB


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 313, 9561,  167,  188,  359,   95,   66,   16,   29,    1,    1,
           0,    0,    0,    0]])>

In [49]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

len(words_in_vocab), top_5_words, bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['package', 'pacing', 'pacifics', 'pacic', 'p45perez'])

## Create an Embedding layer

In [50]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(
    input_dim=max_vocab_length,
    output_dim=128,
    input_length=max_length,
)


In [51]:
sample = random.choice(train_tweets)

print(sample)
embedding(text_vectorizer([sample]))


Time collapse is such a cool video technique.  https://t.co/upLFSqMr0C


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04235158,  0.02281192, -0.02545384, ..., -0.04779514,
         -0.00759042, -0.01136395],
        [ 0.04359404, -0.02315238,  0.04942812, ...,  0.02220322,
         -0.0411651 ,  0.03408865],
        [-0.03728552, -0.00727134,  0.04579153, ...,  0.02998617,
          0.04437393,  0.01135564],
        ...,
        [ 0.02205273, -0.03378798,  0.04782547, ...,  0.01508626,
          0.04487285,  0.04982967],
        [ 0.02205273, -0.03378798,  0.04782547, ...,  0.01508626,
          0.04487285,  0.04982967],
        [ 0.02205273, -0.03378798,  0.04782547, ...,  0.01508626,
          0.04487285,  0.04982967]]], dtype=float32)>

## Modelling Experiments:

0. Naive Bayes w/ TF-IDF encoder
1. Feed-forward neural network
2. RNN - LSTM
3. RNN - GRU
4. Bidirectional RNN
5. 1D Convolutional Neural Network
6. TFHub Pre-trained Feature Extractor




### Model 0: Multinomial Naive Bayes with TF-IDF Vectorizer

In [90]:
def display_overall_metrics(y_true, y_pred):
    """
    Displays overall metrics for the predictions.
    These include:
        - Accuracy
        - F1 Score
        - Precision
        - Recall

    :param y_true: 1-d array of y_true labels
    :param y_pred: 1-d array containing predictions
    :return: None
    """

    from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

    print("Accuracy:", round(accuracy_score(y_true, tf.round(y_pred))* 100 , 2))

    print("F1 Score:", round(f1_score(y_true, tf.round(y_pred), average='weighted') * 100, 2))
    print("Precision:", round(precision_score(y_true, tf.round(y_pred), average='weighted')* 100 , 2))
    print("Recall:", round(recall_score(y_true, tf.round(y_pred), average='weighted')* 100 , 2))

In [53]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB()),
])

model_0.fit(train_tweets, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [54]:
baseline_score = model_0.score(val_tweets, val_labels)
baseline_score

0.7979002624671916

In [55]:
baseline_preds = model_0.predict(val_tweets)

In [97]:
baseline_preds

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [98]:
display_overall_metrics(baseline_preds, val_labels,)

Accuracy: 79.79
F1 Score: 80.68
Precision: 84.29
Recall: 79.79


### Model 1: Feed-Forward Neural Network

In [73]:
import tensorflow as tf
from tensorflow.keras import layers

inputs = layers.Input(shape = (1, ), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)

model_1 = tf.keras.Model(inputs, outputs)

In [74]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 1280129 (

In [76]:
model_1.compile(loss = 'binary_crossentropy',
                optimizer = 'Adam',
                metrics = ['accuracy'])

In [77]:
model_1.fit(x = train_tweets,
            y = train_labels,
            epochs = 5,
            validation_data = (val_tweets, val_labels))

Epoch 1/5
215/215 [==============================] - 10s 35ms/step - loss: 0.6124 - accuracy: 0.6941 - val_loss: 0.5409 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4421 - accuracy: 0.8194 - val_loss: 0.4800 - val_accuracy: 0.7900
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3486 - accuracy: 0.8622 - val_loss: 0.4663 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2863 - accuracy: 0.8888 - val_loss: 0.4733 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2400 - accuracy: 0.9095 - val_loss: 0.4885 - val_accuracy: 0.7861


In [78]:
model_1.evaluate(val_tweets, val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 0.4885 - accuracy: 0.7861


[0.48850584030151367, 0.7860892415046692]

In [99]:
preds_1 = tf.round(tf.squeeze(model_1.predict(val_tweets)))

24/24 [==============================] - 0s 6ms/step


In [101]:
display_overall_metrics(preds_1, val_labels)

Accuracy: 78.61
F1 Score: 78.82
Precision: 79.33
Recall: 78.61


### Project the text embeddings to visualize them better

In [105]:
weights = model_1.get_layer('embedding_1').get_weights()[0]
vocab = text_vectorizer.get_vocabulary()

In [106]:
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [107]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Model 2: Recurrent Neural Network: LSTM (Long Short-Term Memory)

In [117]:
import tensorflow as tf
from tensorflow.keras import layers

inputs = layers.Input(shape = (1, ), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences = False)(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)

model_2 = tf.keras.Model(inputs, outputs)

In [118]:
model_2.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_4 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [119]:
model_2.compile(loss = 'binary_crossentropy',
                optimizer = 'Adam',
                metrics = ['accuracy'])

In [120]:
model_2.fit(x = train_tweets,
            y = train_labels,
            epochs = 5,
            validation_data = (val_tweets, val_labels))

Epoch 1/5
215/215 [==============================] - 11s 40ms/step - loss: 0.2165 - accuracy: 0.9240 - val_loss: 0.5723 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1529 - accuracy: 0.9437 - val_loss: 0.6289 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1275 - accuracy: 0.9531 - val_loss: 0.6873 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1053 - accuracy: 0.9603 - val_loss: 0.7893 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0850 - accuracy: 0.9670 - val_loss: 0.8614 - val_accuracy: 0.7756


In [121]:
preds_2 = tf.round(tf.squeeze(model_2.predict(val_tweets)))

24/24 [==============================] - 1s 4ms/step


In [122]:
display_overall_metrics(preds_2, val_labels)

Accuracy: 77.56
F1 Score: 77.65
Precision: 77.8
Recall: 77.56


In [123]:
display_overall_metrics(baseline_preds, val_labels)

Accuracy: 79.79
F1 Score: 80.68
Precision: 84.29
Recall: 79.79


### Model 3: Recurrent Neural Network: GRU (Gated Recurrent Unit)

In [128]:
import tensorflow as tf
from tensorflow.keras import layers

inputs = layers.Input(shape = (1, ), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)

model_3 = tf.keras.Model(inputs, outputs)

model_3.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru_2 (GRU)                 (None, 64)                37248     
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [129]:
model_3.compile(loss = 'binary_crossentropy',
                optimizer = 'Adam',
                metrics = ['accuracy'])

model_3.fit(x = train_tweets,
            y = train_labels,
            epochs = 5,
            validation_data = (val_tweets, val_labels))

preds_3 = tf.round(tf.squeeze(model_3.predict(val_tweets)))

display_overall_metrics(preds_3, val_labels)

print('\nBaseline:----------\n')
display_overall_metrics(baseline_preds, val_labels)

Epoch 1/5
215/215 [==============================] - 11s 41ms/step - loss: 0.1417 - accuracy: 0.9378 - val_loss: 0.9830 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0664 - accuracy: 0.9755 - val_loss: 0.9899 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0552 - accuracy: 0.9764 - val_loss: 1.1425 - val_accuracy: 0.7493
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0479 - accuracy: 0.9781 - val_loss: 1.2681 - val_accuracy: 0.7480
Epoch 5/5
24/24 [==============================] - 0s 3ms/step
Accuracy: 75.72
F1 Score: 75.76
Precision: 75.82
Recall: 75.72

Baseline:----------

Accuracy: 79.79
F1 Score: 80.68
Precision: 84.29
Recall: 79.79


### Model 4: Bidirectional Recurrent Neural Network

In [131]:
import tensorflow as tf
from tensorflow.keras import layers

inputs = layers.Input(shape = (1, ), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)

model_4 = tf.keras.Model(inputs, outputs)

model_4.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense_10 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1378945 (5.26 MB)
Trainable params: 1378945 (

In [132]:
model_4.compile(loss = 'binary_crossentropy',
                optimizer = 'Adam',
                metrics = ['accuracy'])

model_4.fit(x = train_tweets,
            y = train_labels,
            epochs = 5,
            validation_data = (val_tweets, val_labels))

print('\n')
preds_4 = tf.round(tf.squeeze(model_4.predict(val_tweets)))

display_overall_metrics(preds_4, val_labels)

print('\nBaseline:----------\n')
display_overall_metrics(baseline_preds, val_labels)

Epoch 1/5
215/215 [==============================] - 13s 39ms/step - loss: 0.0981 - accuracy: 0.9711 - val_loss: 0.9921 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0449 - accuracy: 0.9797 - val_loss: 1.3513 - val_accuracy: 0.7441
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0418 - accuracy: 0.9788 - val_loss: 1.4449 - val_accuracy: 0.7480
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0386 - accuracy: 0.9810 - val_loss: 1.1178 - val_accuracy: 0.7546
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0361 - accuracy: 0.9818 - val_loss: 1.4917 - val_accuracy: 0.7756


24/24 [==============================] - 1s 3ms/step
Accuracy: 77.56
F1 Score: 77.67
Precision: 77.89
Recall: 77.56

Baseline:----------

Accuracy: 79.79
F1 Score: 80.68
Precision: 84.29
Recall: 79.79


### Model 5: 1-Dimensional Convolutional Neural Network

In [137]:
import tensorflow as tf
from tensorflow.keras import layers

inputs = layers.Input(shape = (1, ), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(64, 4)(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)

model_5 = tf.keras.Model(inputs, outputs)

model_5.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_3 (Conv1D)           (None, 12, 64)            32832     
                                                                 
 global_average_pooling1d_2  (None, 64)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_13 (Dense)            (None, 1)                 65 

In [138]:
model_5.compile(loss = 'binary_crossentropy',
                optimizer = 'Adam',
                metrics = ['accuracy'])

model_5.fit(x = train_tweets,
            y = train_labels,
            epochs = 5,
            validation_data = (val_tweets, val_labels))

print('\n')
preds_5 = tf.round(tf.squeeze(model_5.predict(val_tweets)))

display_overall_metrics(preds_5, val_labels)

print('\nBaseline:----------\n')
display_overall_metrics(baseline_preds, val_labels)

Epoch 1/5
215/215 [==============================] - 10s 36ms/step - loss: 0.1243 - accuracy: 0.9581 - val_loss: 0.9945 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0783 - accuracy: 0.9705 - val_loss: 1.0925 - val_accuracy: 0.7428
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0658 - accuracy: 0.9752 - val_loss: 1.2117 - val_accuracy: 0.7402
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0601 - accuracy: 0.9750 - val_loss: 1.2501 - val_accuracy: 0.7349
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0582 - accuracy: 0.9764 - val_loss: 1.2656 - val_accuracy: 0.7402


24/24 [==============================] - 0s 3ms/step
Accuracy: 74.02
F1 Score: 74.02
Precision: 74.02
Recall: 74.02

Baseline:----------

Accuracy: 79.79
F1 Score: 80.68
Precision: 84.29
Recall: 79.79


### Model 6: TFHub Pre-Trained Feature Extractor

In [154]:
import tensorflow_hub as hub

encoder = hub.KerasLayer(
    "https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2",
    input_shape = (),
    dtype = tf.string,
)

model_6 = tf.keras.Sequential([
    encoder,
    layers.Dense(128, activation = 'sigmoid'),
    layers.Dense(64, activation = 'sigmoid'),
    layers.Dense(1, activation = 'sigmoid')
])

model_6.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_9 (KerasLayer)  (None, 512)               256797824 
                                                                 
 dense_29 (Dense)            (None, 128)               65664     
                                                                 
 dense_30 (Dense)            (None, 64)                8256      
                                                                 
 dense_31 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256871809 (979.89 MB)
Trainable params: 73985 (289.00 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [155]:
model_6.compile(loss = 'binary_crossentropy',
                optimizer = 'Adam',
                metrics = ['accuracy'])

model_6.fit(x = train_tweets,
            y = train_labels,
            epochs = 5,
            validation_data = (val_tweets, val_labels))

print('\n')
preds_6 = tf.round(tf.squeeze(model_6.predict(val_tweets)))

display_overall_metrics(preds_6, val_labels)

print('\nBaseline:----------\n')
display_overall_metrics(baseline_preds, val_labels)

Epoch 1/5
215/215 [==============================] - 7s 14ms/step - loss: 0.6077 - accuracy: 0.6898 - val_loss: 0.4919 - val_accuracy: 0.7979
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4447 - accuracy: 0.8043 - val_loss: 0.4663 - val_accuracy: 0.7927
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4201 - accuracy: 0.8151 - val_loss: 0.4428 - val_accuracy: 0.8058
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4125 - accuracy: 0.8186 - val_loss: 0.4423 - val_accuracy: 0.8058
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4059 - accuracy: 0.8213 - val_loss: 0.4360 - val_accuracy: 0.8097


24/24 [==============================] - 1s 9ms/step
Accuracy: 80.97
F1 Score: 81.11
Precision: 81.42
Recall: 80.97

Baseline:----------

Accuracy: 79.79
F1 Score: 80.68
Precision: 84.29
Recall: 79.79
